## Customer Churn Model Scoring
#### The objectives of this lab is:
- score **new** customer data against a pre-built model
- schedule the notebook to run via the Notebook scheduler

### Step 1: Download new customer data

In [ ]:
import wget
url_customer='https://raw.githubusercontent.com/SidneyPhoon/IntroToNotebooksLab/master/data/new_customer_data.csv'

#remove existing files before downloading
!rm -f new_customer_data.csv

customerFilename=wget.download(url_customer)

!ls -l new_customer_data.csv

### Step 2: Read data into a Spark DataFrame
**Note**: the new dataset does not contain the label column

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

newData= spark.read\
  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
  .option('header', 'true')\
  .option("inferSchema", "true")\
  .load(customerFilename)

In [ ]:
newData = newData.withColumnRenamed("Est Income", "EstIncome").withColumnRenamed("Car Owner","CarOwner")
newData.toPandas().head()

### Step 3: Load Saved Model

In [ ]:
from pyspark.ml import PipelineModel
model1_loaded = PipelineModel.load("PredictChurn.churnModel")

### Step 4: Score the new data
Note: The scored output contains the predicted values and confidence scores

In [ ]:
result = model1_loaded.transform(newData)

### Step 5: Export Score into a csv file

In [ ]:
#Select ID, prediction and probability fields from the result dataframe

r1=result.select(result["ID"],result["predictedLabel"],result["prediction"],result["probability"])
r1.show(5,False)

#### Decompose the probability column
The probability column contains a vector for each record, and the elements must be extracted

In [ ]:
from pyspark.sql import Row
from pyspark.sql.types import DoubleType
from pyspark.sql.functions import udf
from pyspark.ml.linalg import Vectors

udf_0 = udf(lambda vector: float(vector[0]), DoubleType())
udf_1 = udf(lambda vector: float(vector[1]), DoubleType())

r2 = (r1.select(r1["ID"], r1["prediction"],r1["probability"])
    .withColumn('probability_0', udf_0(r1.probability))
    .withColumn('probability_1', udf_1(r1.probability))
    .drop("probability"))

r2.show(10, False)

#### Connect to Object Storage
In order to write the scores to Object Storage, specify the credentials to connect to your instance of Object Storage.  The easiet way to do that is:
- If you do not already have a file in Object Storage, load a file into it using the **Files** interface
- Choose "*Insert SparkSession DataFame*" to generate the credentials and code to connect to Object Storage

![Load Files](https://raw.githubusercontent.com/yfphoon/IntroToNotebooks/master/images/upload_files.png)

- Edit the code to comment out or edit the code that reads the file.  The edited code cell should look like this

![credentials](https://raw.githubusercontent.com/yfphoon/IntroToNotebooks/master/images/generated_credentials.png)



In [ ]:
# insert code here



#### Write sores .csv file

The code cell below specifies the options for saving the csv file.  Check that you have specified the **TARGET_CONTAINER** to point to your project.

In [ ]:
from ingest.Connectors import Connectors

objectstoresaveOptions = {
        Connectors.BluemixObjectStorage.AUTH_URL          : credentials['auth_url'],
        Connectors.BluemixObjectStorage.USERID            : credentials['user_id'],
        Connectors.BluemixObjectStorage.PASSWORD          : credentials['password'],
        Connectors.BluemixObjectStorage.PROJECTID         : credentials['project_id'],
        Connectors.BluemixObjectStorage.REGION            : credentials['region'],
        Connectors.BluemixObjectStorage.TARGET_CONTAINER  : 'IntroToNotebooks',
        Connectors.BluemixObjectStorage.TARGET_FILE_NAME  : 'churn_scores.csv',
        Connectors.BluemixObjectStorage.TARGET_WRITE_MODE : 'write'}


r2.write.format("com.ibm.spark.discover").options(**objectstoresaveOptions).save()

In [ ]:
r3 = spark.read\
  .format('csv')\
  .load(bmos.url('IntroToNotebooks', 'churn_scores.csv'))
r3.select(r3["_c0"].alias("ID"), r3["_c1"].alias("prediction"), r3["_c2"].alias("probability_0"), r3["_c3"].alias("probability_1")).show(5, False)

### Step 6: Schedule this notebook to run at a time and frequency of your choice
Click on the "clock" icon at the top right

You have come to the end of this notebook

** Sidney Phoon** <br/>
yfphoon@us.ibm.com<br/>
September 1st, 2017